In [1]:
%pip install transformers datasets evaluate torch nltk rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=55941017a9d440fdedd2e251f24e5521a4c715648cac8c41665c0063be47e863
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


## Directly imported Dataset From Huggingface

In [2]:
# from datasets import Dataset, DatasetDict
# from datasets import load_dataset
# dataset = load_dataset("toughdata/quora-question-answer-dataset")
# dataset = dataset["train"].train_test_split(test_size=0.2)

In [3]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 5.6 MB/s eta 0:00:00


## Preprocessed Dataset

In [4]:
import pandas as pd

# Load preprocessed datasets
df_test_preprocessed = pd.read_csv('/kaggle/input/quora-quad-preprocessed/preprocessed_test_dataset.csv')
df_train_preprocessed = pd.read_csv('/kaggle/input/quora-quad-preprocessed/preprocessed_train_dataset.csv')

from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(df_train_preprocessed)
test_dataset = Dataset.from_pandas(df_test_preprocessed)
dataset_preprocessed= DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})


print("Datasets converted back to original format for model training.")
from datasets import Dataset, DatasetDict

# Ensure 'question' and 'answer' columns are of type string
df_train_preprocessed['question'] = df_train_preprocessed['question'].astype(str)
df_train_preprocessed['answer'] = df_train_preprocessed['answer'].astype(str)

df_test_preprocessed['question'] = df_test_preprocessed['question'].astype(str)
df_test_preprocessed['answer'] = df_test_preprocessed['answer'].astype(str)

train_dataset = Dataset.from_pandas(df_train_preprocessed)
test_dataset = Dataset.from_pandas(df_test_preprocessed)

dataset_preprocessed = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

print(dataset_preprocessed)

Datasets converted back to original format for model training.
DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 44258
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 11187
    })
})


In [5]:
import nltk
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import BartTokenizer, DataCollatorForSeq2Seq
from transformers import BartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

# Ensure GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the tokenizer and model for BART
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large").to(device)

# Load the data collator for BART
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


2024-07-30 14:44:02.415318: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 14:44:02.415433: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 14:44:02.543707: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
from datasets import Dataset
import numpy as np
import nltk
import evaluate
from transformers import BartTokenizer, BartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
from sklearn.metrics import precision_recall_fscore_support

# Define fraction of the dataset to use
fraction = 0.01  # Use 10% of the dataset

# Create smaller subsets of the dataset
train_size = int(len(dataset_preprocessed["train"]) * fraction)
test_size = int(len(dataset_preprocessed["test"]) * fraction)

# Shuffle and select the subset
train_subset = dataset_preprocessed["train"].shuffle(seed=42).select(range(train_size))
test_subset = dataset_preprocessed["test"].shuffle(seed=42).select(range(test_size))

prefix = "answer the question: "

def preprocess_function(examples):
    """Add prefix to the sentences, tokenize the text, and set the labels"""
    inputs = [prefix + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    
    labels = tokenizer(text_target=examples["answer"], max_length=512, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Load BART tokenizer and model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")

# Tokenize the subset datasets
tokenized_train_dataset = train_subset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_subset.map(preprocess_function, batched=True)

# Set up ROUGE, BLEU, Precision, Recall, and F1 score for evaluation
nltk.download("punkt", quiet=True)
rouge_metric = evaluate.load("rouge")
bleu_metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # ROUGE expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Compute ROUGE score
    rouge_result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Compute BLEU score
    decoded_preds_for_bleu = [" ".join(pred.split()) for pred in decoded_preds]
    decoded_labels_for_bleu = [[" ".join(label.split())] for label in decoded_labels]
    bleu_result = bleu_metric.compute(predictions=decoded_preds_for_bleu, references=decoded_labels_for_bleu)

    # Compute Precision, Recall, and F1 on sentence level
    true_labels = [label.split() for label in decoded_labels]
    pred_labels = [pred.split() for pred in decoded_preds]

    # Flatten the lists
    true_labels_flat = [token for sublist in true_labels for token in sublist]
    pred_labels_flat = [token for sublist in pred_labels for token in sublist]
    
    # Ensure the lengths match by truncating to the minimum length
    min_length = min(len(true_labels_flat), len(pred_labels_flat))
    true_labels_flat = true_labels_flat[:min_length]
    pred_labels_flat = pred_labels_flat[:min_length]

    precision, recall, f1, _ = precision_recall_fscore_support(true_labels_flat, pred_labels_flat, average='weighted', zero_division=0)

    # Combine metrics
    combined_result = {
        **rouge_result,
        "bleu": bleu_result["score"],
        "precision": precision,
        "recall": recall,
        "f1": f1
    }
    return combined_result

# Set up training arguments with logging configuration
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     logging_strategy="steps",
#     logging_steps=50,  # Adjust the logging frequency as needed
#     learning_rate=3e-4,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=4,
#     weight_decay=0.01,
#     save_total_limit=3,
#     num_train_epochs=2,
#     predict_with_generate=True,
#     push_to_hub=False,
#     report_to="none",  # You can also set this to "tensorboard" if using TensorBoard
# )
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,  # Increase logging frequency if needed
    learning_rate=5e-5,  # Adjust learning rate
    per_device_train_batch_size=8,  # Adjust batch size
    per_device_eval_batch_size=4,  # Adjust batch size
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,  # Increase number of epochs
    predict_with_generate=True,
    push_to_hub=False,
    report_to="none",  # Use "tensorboard" if needed
)

# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


Map:   0%|          | 0/442 [00:00<?, ? examples/s]

Map:   0%|          | 0/111 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu,Precision,Recall,F1
1,No log,5.969712,0.088763,0.023697,0.082052,0.082058,0.003844,0.004353,0.001088,0.001741
2,6.519000,5.958683,0.066584,0.015152,0.061768,0.062606,0.002152,0.000000,0.000000,0.000000
3,6.519000,5.867538,0.083761,0.021389,0.075105,0.075429,0.007906,0.001875,0.002000,0.001916
4,5.520200,5.874949,0.097157,0.020737,0.089584,0.090122,0.003980,0.004756,0.002217,0.002273
5,5.520200,5.873425,0.093770,0.019564,0.086462,0.086930,0.002340,0.000344,0.001171,0.000532


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=280, training_loss=5.758419908796038, metrics={'train_runtime': 268.8748, 'train_samples_per_second': 8.219, 'train_steps_per_second': 1.041, 'total_flos': 123389518282752.0, 'train_loss': 5.758419908796038, 'epoch': 5.0})

In [1]:
trainer.save_model('./results/final_model')

NameError: name 'trainer' is not defined